In [1]:
import subprocess
import json

In [2]:
def regex_equiv(gold, predicted):
    if gold == predicted:
        return True
    try:
        out = subprocess.check_output(['java', '-jar', 'regex_dfa_equals.jar', '{}'.format(gold), '{}'.format(predicted)])
#          print("out: {}".format(out))
        #print(out.decode('utf-8'))
        if '\n1' in out.decode('utf-8'):
            return True
        else:
            return False
    except Exception as e:
        return False


In [ ]:

names = ["Phi_Raw_Output","Phi_Refined_Output"]
for name in names:
    print("Starting: ", name)
    filename = f'../Generation/Output/{name}.json'
    outfilename = f'./Output/{name}_DFA_Result.json'

    with open(filename, 'r') as f:
        data = json.load(f)
    print(len(data))

    dfa_result = []
    for item in data:
        print("Starting item: ", item['id'])
        id = item['id']
        gold_expression = item['expression']
        # print ("Expression: ", gold_expression)
        for i in range(len(item['phi_output'])):
            predicted_expression = item['phi_output'][i]['text']
            item['completion'] = predicted_expression
            passed = regex_equiv(gold_expression, predicted_expression)
            
            dfa_result.append({
                'id': id,
                'completion': predicted_expression,
                'passed': passed
            })  
        print("Ending item: ", item['id'])

    with open(outfilename, 'w') as f:
        json.dump(dfa_result, f, indent=4)

In [ ]:
# Write a code for pass@k
def pass_at_k(results, ks):
    total = len(results)
    for k in ks:
        correct = 0
        for result in results.values():
            passed = [r["passed"] for r in result[:k]]
            correct += (1 if sum(passed)!=0 else 0)
        print(f"DFA-EQ@{k}: {(correct/total)*100}")

        

    

In [ ]:
# names = ["Text_DaVinci_Raw_Output","Text_DaVinci_Refined_Output","GPT3.5_Raw_Output","GPT3.5_Refined_Output", "T5_Raw_Output","T5_Refined_Output"]
names = ["Phi_Raw_Output","Phi_Refined_Output"]
matrix = "DFA"
for name in names:
    print(name)
    filename = f'../Generation/Output/{name}.json'
    outfilename = f'./Output/{name}_{matrix}_Result.json'

    with open(outfilename, 'r') as f:
        result = json.load(f)

    results = defaultdict(list)


    for r in result:
        results[r['id']].append(r)

    pass_at_k(results, [1,3,10])

    # Calculate pass@k.
    total, correct = [], []
    for result in results.values():
        passed = [r["passed"] for r in result]
        total.append(len(passed))
        correct.append(sum(passed))
    total = np.array(total)
    correct = np.array(correct)
    # print(total, correct)
    ks = [1,3,10]
    pass_at_k_result = {f"pass@{k}": (estimate_pass_at_k(total, correct, k).mean())*100
                        for k in ks if (total >= k).all()}
    print(pass_at_k_result)
